# SQL: Permissões

Desde nossas primeiras aulas, realizamos conexões com o **SGBD** (Sistema de Gerenciamento de Bancos de Dados) MySQL, seja no Workbench ou pelos jupyter notebooks.

Abra o arquivo `.env` que utilizamos para configurar as variáveis de ambiente. Provavelmente você deve encontrar uma linha com este conteúdo:

`MD_DB_USERNAME="root"`

Bem, isto significa que estamos nos conectando utilizando o usuário `root`!

**<div id="user_root"></div>**
**O que é o usuário root no MySQL?**

O usuário `root` é um **superusuário** no MySQL. Ele tem acesso total ao SGBD, funcionando como um usuário administrador. Possui privilégios ilimitados para criar, modificar e excluir bancos de dados, bem como conceder ou revogar permissões a outros usuários. 

Ele é criado durante a instalação do MySQL e é altamente recomendado que sejam tomadas medidas de segurança para proteger essa conta, como definir uma senha forte e restringir o acesso remoto apenas aos endereços IP autorizados.

## E a aplicação em produção?!

Estamos desenhando soluções de banco de dados que um dia serão entregues a algum cliente (estarão em produção, prontas para serem integradas a outros sistemas). Quando a conexão com o SGBD for exposta para uso por alguma API (como a que desenvolveram no projeto), vamos precisar de um usuário e senha para conexão com o MySQL.

A resposta em <a href="#user_root">"O que é o usuário root no MySQL?"</a> já dá uma ideia que utilizar o usuário `root` nestas situações não parece correto. Caso ocorra algum vazamento da senha ou *SQL injection*, a base poderia ficar exposta tanto para leitura quanto para escrita, ou seja, um hacker poderia tanto **ver os dados** nas tabelas quanto **editar** e **exluir**.

## Qual a alternativa?!

Uma alternativa mais adequada envolve:

- **Criar usuários únicos**: vamos criar múltiplos usuário com acesso ao banco. Cada indivíduo ou aplicação em *deploy* terá seu usuário e senha personalizado (sem compartilhamento de senhas - Netflix vibes!)


- **Dar permissões**: vamos adicionar ao usuário apenas as permissões necessárias para que seu papel seja cumprido. Por exemplo, um colaborador de uma consultoria externa que presta serviços à uma empresa precisa ter acesso à base toda? Não! Ele provavelmente precisará apenas ler dados e não escrever, além de ser indicado que tenha permissão de visualização apenas em parte das tabelas (as que forem úteis para desenvolvimento do projeto).

Antes de praticar, vamos importas as bibliotecas necessárias

In [1]:
import mysql.connector
from functools import partial
import os
import insperautograder.jupyter as ia
from dotenv import load_dotenv

load_dotenv()

def run_query(connection, query, args=None):
    """
    Função que recebe uma conexão, query e argumentos;
    executa a query na conexão, e retorna os resultados
    """
    with connection.cursor() as cursor:
        print("Executando query:")
        cursor.execute(query, args)
        for result in cursor:
            print(result)

def get_connection_helper():

    connection = mysql.connector.connect(
        host=os.getenv("MD_DB_SERVER"),
        user=os.getenv("MD_DB_USERNAME"),
        password=os.getenv("MD_DB_PASSWORD"),
        database="fiscamuni",
    )
    return connection, partial(run_query, connection)

## Criar usuário no MySQL

Podemos criar usuário no MySQL com a seguinte sintaxe:

```MySQL
CREATE USER '<user>'@'<host>' IDENTIFIED BY '<password>';
```

Vamos experimentar! Abra o Workbench e execute:

```MySQL
CREATE USER 'joao'@'localhost' IDENTIFIED BY 'abc123';
```

Então, vamos tentar fazer uma conexão com o novo usuário. A dinâmica da aula será utilizar o Workbench com o usuário `root` e os notebooks com os usuário criados para testar as permissões.

<div class="alert alert-warning">

Provavelmente você verá em alguns materiais a indicação do uso do comando
```SQL
FLUSH PRIVILEGES;
```

<br>
após alterar permissões de um usuário. Ele força a atualização das permissões pelo servidor. Os `GRANTS` tomam efeito assim que executados, mas o uso do `FLUSH` pode ser necessário em alguns casos.
</div>

In [2]:
conn1 = mysql.connector.connect(
    host="localhost",
    user="joao",
    password="abc123",
    database="musica"
)

ProgrammingError: 1045 (28000): Access denied for user 'joao'@'172.17.0.1' (using password: YES)

Perceba que a conexão falhou! Isto porque, apesar do usuário ter sido criado, ele não possui as permissões necessárias!

## Dar Permissão

Para alterar permissões de um usuário, iremos utilizar `GRANT`.

A seguinte sintaxe resumida pode ser utilizada:

```mysql
GRANT PRIVILEGE ON database.table TO 'username'@'host';
```

Veja mais em https://dev.mysql.com/doc/refman/8.0/en/grant.html

Abra o Workbench e execute:

```mysql
GRANT SELECT ON musica.* TO 'joao'@'localhost';
```

Assim, o usuário terá a permissão de **SELECT** em todas as tabelas do database `musica`. Vamos testar!

In [6]:
conn1 = mysql.connector.connect(
    host="localhost",
    user="joao",
    password="abc123",
    database="musica"
)

In [7]:
run_query(conn1, "SELECT * FROM AUTOR")

Executando query:
(1, 'Renato Russo')
(2, 'Tom Jobim')
(3, 'Chico Buarque')
(4, 'Dado Villa-Lobos')
(5, 'Marcelo Bonfá')
(6, 'Ico Ouro-Preto')
(7, 'Vinicius de Moraes')
(8, 'Baden Powell')
(9, 'Paulo Cesar Pinheiro')
(10, 'João Bosco')
(11, 'Aldir Blanc')
(12, 'Joyce')
(13, 'Ana Terra')
(14, 'Cartola')
(15, 'Cláudio Tolomei')
(16, 'João Nogueira')
(17, 'Suely Costa')
(18, 'Guinga')
(19, 'Danilo Caymmi')
(20, 'Tunai')
(21, 'Sérgio Natureza')
(22, 'Heitor Villa Lobos')
(23, 'Ferreira Gullar')
(24, 'Catulo da Paixão Cearense')
(25, 'Zezé di Camargo')
(26, 'Niltinho Edilberto')
(27, 'Marisa Monte')
(28, 'Carlinhos Brown')
(29, 'Gonzaga Jr')
(30, 'Roberto Mendes')
(31, 'Ana Basbaum')
(32, 'Caetano Veloso')
(33, 'José Miguel Wisnik')
(34, 'Vevé Calazans')
(35, 'Gerônimo')
(36, 'Sérgio Natureza')
(37, 'Roberto Carlos')
(38, 'Erasmo Carlos')
(39, 'Renato Teixeira')
(40, 'Chico César')
(41, 'Vanessa da Mata')
(42, 'Jorge Portugal')
(43, 'Lilian Knapp')
(44, 'Renato Barros')
(45, 'Bebel Gilberto

In [8]:
run_query(conn1, "SELECT * FROM CD")

Executando query:
(1, 1, 'Mais do Mesmo', 15, datetime.date(1998, 10, 1), 5)
(2, 2, 'Bate-Boca', 12, datetime.date(1999, 7, 1), 3)
(3, 3, 'Elis Regina - Essa Mulher', 13, datetime.date(1989, 5, 1), 1)
(4, 2, 'A Força que nunca Seca', 14, datetime.date(1998, 12, 1), 1)
(5, 3, 'Perfil', 11, datetime.date(2001, 5, 1), 2)
(6, 2, 'Barry Manilow Greatest Hits Vol I', 10, datetime.date(1991, 11, 1), 7)
(7, 2, 'Listen Without Prejudice', 9, datetime.date(1991, 10, 1), None)


Pronto! Agora a conexão está funcionando! Vamos fechar a conexão...

In [9]:
# Não vamos manter a conexão aberta!
conn1.close()

... e tentar realizar a conexão com a base de dados `tranqueira`:

In [10]:
conn2 = mysql.connector.connect(
    host="localhost",
    user="joao",
    password="abc123",
    database="tranqueira"
)

ProgrammingError: 1044 (42000): Access denied for user 'joao'@'172.17.0.1' to database 'tranqueira'

Obtivemos novamente um erro. Caso o usuário `joao` realmente necessite acesso à base `tranqueira`, um novo `GRANT` deve ser realizado!

Ao fazer *deploy* de aplicações em produção, é indicado ter usuários diferentes por aplicação, apenas as permissões necessárias.

Vamos criar um segundo usuário com permissão de `SELECT` na base `tranqueira`. Execute o comando no Workbench:
    
```mysql
CREATE USER 'ana'@'localhost' IDENTIFIED BY '456456';
GRANT SELECT ON tranqueira.* TO 'ana'@'localhost';
```
Teste a conexão:

In [ ]:
conn3 = mysql.connector.connect(
    host="localhost",
    user="ana",
    password="456456",
    database="tranqueira"
)

Vamos ver se conseguimos ler a tabela `perigo`:

In [ ]:
run_query(conn3, "SELECT * FROM perigo")

E fazer um **INSERT**, também na tabela `perigo`:

In [ ]:
sql = """
INSERT INTO `tranqueira`.`perigo`
    (`id`,`Nome`)
VALUES
    (10,'Moral')
"""

run_query(conn3, sql);

Novamente, precisamos dar permissão de inserção! Dê a permissão pelo Workbench e teste novamente! Perceba que agora a permissão será para **apenas uma tabela**:

```mysql
GRANT INSERT ON tranqueira.perigo TO 'ana'@'localhost';
```

In [ ]:
conn3.rollback()

In [ ]:
sql = """
INSERT INTO `tranqueira`.`perigo`
    (`id`, `Nome`)
VALUES
    (10,'Moral')
"""

run_query(conn3, sql);

Vamos verificar se realmente conseguimos inserir:

In [ ]:
run_query(conn3, "SELECT * FROM perigo")

Então, desfazemos a inserção e fechamos a conexão!

In [ ]:
conn3.rollback()
conn3.close()

<div class="alert alert-warning">

Algums exemplos de GRANTS!
    
```mysql
GRANT ALL PRIVILEGES ON *.* TO 'maria'@'localhost' WITH GRANT OPTION;
GRANT CREATE TEMPORARY TABLES ON coemu.* TO 'user_deploy_api'@'localhost';
GRANT EXECUTE ON sys.* TO 'user_dashboard'@'localhost' WITH GRANT OPTION;
GRANT SELECT, SHOW VIEW ON cartracking.* TO 'leitor'@'localhost';
```
</div>

## Revogar Permissão

Para ver as permissões de um usuário, utilize:

```mysql
SHOW GRANTS FOR 'ana'@'localhost';
```


Caso queira revogar permissões de um usuário, iremos utilizar `REVOKE`.

```mysql
REVOKE SELECT ON tranqueira.* FROM 'ana'@'localhost';
```

In [ ]:
conn4 = mysql.connector.connect(
    host="localhost",
    user="ana",
    password="456456",
    database="tranqueira"
)

Conseguimos fazer o login. Vamos testar se o **SELECT** funciona!

In [ ]:
run_query(conn4, "SELECT * FROM perigo;")

Vamos analisar os grants restantes ao usuário:

In [ ]:
run_query(conn4, "SHOW GRANTS FOR 'ana'@'localhost';")

Temos o `GRANT INSERT` mas não mais o `GRANT SELECT`, então nossa única permissão é para inserir linhas na tabela `perigo`.

In [ ]:
conn4.close()

Remova também esta permissão:

```mysql
REVOKE INSERT ON tranqueira.perigo FROM 'ana'@'localhost';
```

E teste novamente:

In [ ]:
conn5 = mysql.connector.connect(
    host="localhost",
    user="ana",
    password="456456",
    database="tranqueira"
)

## Hosts

Ao criar um usuário no MySQL, podemos especificar de qual **host** ele pode se conectar. Nos exemplos que apresentamos, o usuário `joao` só poderá se conectar ao MySQL do `localhost`.

Para permitir que o usuário se conecte de outros hosts, devemos criar um novo usuário com o mesmo nome e senha, mas com uma configuração de `host` diferente.

Por exemplo, para permitir que o usuário `joao` se conecte de qualquer host, você pode criar um novo usuário com a seguinte instrução SQL:

```mysql
CREATE USER 'joao'@'%' IDENTIFIED BY 'abc123';
```

No exemplo acima, o símbolo `%` indica que o usuário pode se conectar de qualquer host.

Então, basta condeder as permissões adequadas. Se quiser todas as permissões para o usuário "joao" ao banco de dados `tranqueira`, execute a seguinte instrução SQL:

```mysql
GRANT ALL PRIVILEGES ON tranqueira.* TO 'joao'@'%';
```

## Exercícios para entrega

Esta aula tem atividade para entrega, confira os prazos e exercícios

In [11]:
ia.tasks()

|    | Atividade            | De                  | Até                 |
|---:|:---------------------|:--------------------|:--------------------|
|  0 | newborn              | 2025-02-01 00:00:00 | 2025-05-30 00:00:00 |
|  1 | select01             | 2025-02-06 16:00:00 | 2025-02-15 23:59:59 |
|  2 | ddl                  | 2025-02-20 00:00:00 | 2025-02-27 23:59:59 |
|  3 | dml                  | 2025-02-24 00:00:00 | 2025-03-06 15:45:59 |
|  4 | agg_join             | 2025-02-26 00:00:00 | 2025-03-09 23:59:59 |
|  5 | group_having         | 2025-03-06 00:00:00 | 2025-03-13 23:59:59 |
|  6 | views                | 2025-03-10 00:00:00 | 2025-03-16 23:59:59 |
|  7 | sql_review1          | 2025-03-13 00:00:00 | 2025-03-20 23:59:59 |
|  8 | permissions          | 2025-03-20 00:00:00 | 2025-04-07 23:59:59 |
|  9 | ai_md_23_1           | 2025-03-23 00:00:00 | 2025-03-30 23:59:59 |
| 10 | ai_md_23_2           | 2025-03-23 00:00:00 | 2025-03-30 23:59:59 |
| 11 | ai_md_25_1           | 2025-03-31 00:00:00 | 2025-03-31 10:05:00 |
| 12 | desafio_normalizacao | 2025-04-07 00:00:00 | 2025-05-30 00:00:00 |

In [12]:
ia.grades(task="permissions")

|    | Atividade   | Exercício   |   Peso |   Nota |
|---:|:------------|:------------|-------:|-------:|
|  0 | permissions | ex01        |      1 |      0 |
|  1 | permissions | ex02        |      1 |      0 |
|  2 | permissions | ex03        |      1 |      0 |
|  3 | permissions | ex04        |      1 |      0 |
|  4 | permissions | ex05        |      1 |      0 |
|  5 | permissions | ex06        |      1 |      0 |
|  6 | permissions | ex07        |      1 |      0 |
|  7 | permissions | ex08        |      1 |      0 |
|  8 | permissions | ex09        |      1 |      0 |
|  9 | permissions | ex10        |      1 |      0 |
| 10 | permissions | ex11        |      1 |      0 |
| 11 | permissions | ex12        |      1 |      0 |
| 12 | permissions | ex13b       |      1 |      0 |

Vamos criar nossa tradicional conexão!

In [65]:
root_connection, db = get_connection_helper()

## Base de dados

Utilizaremos a base de dados `fiscamuni`, que busca armazenar informações sobre fiscais e multas aplicadas à propriedades pertecentes a uma empresa.

A base possui o seguinte modelo relacional:

<img src="img/eer_diagram.png">

Execute o script `multas.sql` no Workbench para gerar a base.

**Exercício 1**:

Crie um usuário `camilaw2` com login a partir do `localhost` e senha `699a1deacb58`.

In [14]:
sql_ex01 = """
CREATE USER 'camilaw2'@'localhost' IDENTIFIED BY '699a1deacb58'
"""

db(sql_ex01)

Executando query:


In [15]:
ia.sender(answer="sql_ex01", task="permissions", question="ex01", answer_type="pyvar")

interactive(children=(Button(description='Enviar ex01', style=ButtonStyle()), Output()), _dom_classes=('widget…

**Exercício 2**:

Remova o usuário criado no exercício anterior.

In [16]:
sql_ex02 = """
DROP USER 'camilaw2'@'localhost';
"""

db(sql_ex02)

Executando query:


In [17]:
ia.sender(answer="sql_ex02", task="permissions", question="ex02", answer_type="pyvar")

interactive(children=(Button(description='Enviar ex02', style=ButtonStyle()), Output()), _dom_classes=('widget…

**Exercício 3**:

Crie um usuário `marianafag` com login a partir de **qualquer host** e senha `cB18cDd2503F`.

In [18]:
sql_ex03 = """
CREATE USER 'marianafag'@'%' IDENTIFIED BY 'cB18cDd2503F';
"""

db(sql_ex03)

Executando query:


In [19]:
ia.sender(answer="sql_ex03", task="permissions", question="ex03", answer_type="pyvar")

interactive(children=(Button(description='Enviar ex03', style=ButtonStyle()), Output()), _dom_classes=('widget…

**Exercício 4**:

Crie um usuário `pereiradjs` com login a partir do IP `192.168.15.160` e senha `bb3_091#2d6@A70`.

In [20]:
sql_ex04 = """
CREATE USER 'pereiradjs'@'192.168.15.160' IDENTIFIED BY 'bb3_091#2d6@A70';
"""

db(sql_ex04)

Executando query:


In [21]:
ia.sender(answer="sql_ex04", task="permissions", question="ex04", answer_type="pyvar")

interactive(children=(Button(description='Enviar ex04', style=ButtonStyle()), Output()), _dom_classes=('widget…

**Exercício 5**:

Utilize SQL para criar um usuário, observando que:

- O nome de usuário será `diniz`
- A senha será `abc123cba`
- Deve conseguir login de qualquer host

Ainda, o usuário deve ter permissões apenas de:

- **Leitura** na tabela `fiscal`. 

Ou seja, inserção e deleção ou select em outras tabelas ou bases de dados devem estar bloqueadas.

In [30]:
sql_ex05 = """
CREATE USER 'diniz'@'%' IDENTIFIED BY 'abc123cba';
GRANT SELECT ON fiscamuni.fiscal TO 'diniz'@'%';
"""

db(sql_ex05)

Executando query:


DatabaseError: 1396 (HY000): Operation CREATE USER failed for 'diniz'@'%'

In [31]:
ia.sender(answer="sql_ex05", task="permissions", question="ex05", answer_type="pyvar")

interactive(children=(Button(description='Enviar ex05', style=ButtonStyle()), Output()), _dom_classes=('widget…

**Exercício 6**:

Crie um usuário `rem_dash_alu` com login a partir de IPs da **subnet /24** em `192.168.58.0` com senha `9C26189563A7`.

In [40]:
sql_ex06 = """
CREATE USER 'rem_dash_alu'@'192.168.58.%' IDENTIFIED BY '9C26189563A7';
"""

db(sql_ex06)

Executando query:


In [38]:
ia.sender(answer="sql_ex06", task="permissions", question="ex06", answer_type="pyvar")

interactive(children=(Button(description='Enviar ex06', style=ButtonStyle()), Output()), _dom_classes=('widget…

**Exercício 7**:

Suponha que um exista um usuário `marianatt` com permissão de login de **qualquer host**.

Altere a senha do usuário para `b2a8b85f76b1b923`.

In [43]:
sql_ex07 = """
ALTER USER 'marianatt'@'%' IDENTIFIED BY 'b2a8b85f76b1b923';
"""

db(sql_ex07)

Executando query:


DatabaseError: 1396 (HY000): Operation ALTER USER failed for 'marianatt'@'%'

In [44]:
ia.sender(answer="sql_ex07", task="permissions", question="ex07", answer_type="pyvar")

interactive(children=(Button(description='Enviar ex07', style=ButtonStyle()), Output()), _dom_classes=('widget…

**Exercício 8**:

Escreva uma query que liste:

- O `id_empresa` da empresa.
- A quantidade de propriedades que a empresa possui cadastrada na base. Aqui, a coluna deve se chamar `qtde_propriedades`.
- O valor total de multas da empresa. Aqui, a coluna deve se chamar `valor_total_multas`.

**Obs**:
- Empresas sem propriedades devem ser retornadas com valor zerado (`0`) em `qtde_propriedades`.
- Empresas sem multas devem ser retornadas com valor zerado (`0.00`) em `valor_total_multas`.

Exiba ordenado por:
- `valor_total_multas` (decrescente) e `qtde_propriedades` (decrescente)

In [59]:
sql_ex08 = """
SELECT
    DISTINCT e.id_empresa,
    COALESCE(COUNT(DISTINCT(p.id_propriedade)),0) AS qtde_propriedades,
    COALESCE(SUM(m.valor),0) AS valor_total_multas
FROM empresa AS e
    LEFT JOIN 
    propriedade AS p ON p.id_empresa = e.id_empresa
    LEFT JOIN 
    multa AS m ON m.id_propriedade = p.id_propriedade
GROUP BY 
    e.id_empresa
ORDER BY
    valor_total_multas DESC, qtde_propriedades DESC;

"""

db(sql_ex08)

Executando query:
(2, 3, Decimal('2686.00'))
(7, 4, Decimal('2671.00'))
(5, 1, Decimal('0.00'))
(1, 0, Decimal('0.00'))
(3, 0, Decimal('0.00'))
(4, 0, Decimal('0.00'))
(6, 0, Decimal('0.00'))
(8, 0, Decimal('0.00'))


In [50]:
ia.sender(answer="sql_ex08", task="permissions", question="ex08", answer_type="pyvar")

interactive(children=(Button(description='Enviar ex08', style=ButtonStyle()), Output()), _dom_classes=('widget…

**Exercício 9**:

Transforme a query do exercício anterior em uma **view** chamada `abt_empresa_total`.

Envie apenas o comando de criação da view, não envie `DROP`!

In [60]:
sql_ex09 = """
CREATE VIEW abt_empresa_total AS 
SELECT
    DISTINCT e.id_empresa,
    COALESCE(COUNT(DISTINCT(p.id_propriedade)),0) AS qtde_propriedades,
    COALESCE(SUM(m.valor),0) AS valor_total_multas
FROM empresa AS e
    LEFT JOIN 
    propriedade AS p ON p.id_empresa = e.id_empresa
    LEFT JOIN 
    multa AS m ON m.id_propriedade = p.id_propriedade
GROUP BY 
    e.id_empresa
ORDER BY
    valor_total_multas DESC, qtde_propriedades DESC;
"""

db(sql_ex09)

Executando query:


In [61]:
ia.sender(answer="sql_ex09", task="permissions", question="ex09", answer_type="pyvar")

interactive(children=(Button(description='Enviar ex09', style=ButtonStyle()), Output()), _dom_classes=('widget…

**Exercício 10**:

Sua empresa contratou uma consultoria para atuar em um projeto.

Relações entre empresas sempre expoem problemas de confiança, onde uma não quer que a outra tenha acesso a todos os seus dados.

Assim:
- Crie um usuário `caiomc_consult` com permissão de login de **qualquer host** e senha `6b7997f42e0ebf3a51d2`.
- O consultor deve ter permissão **apenas** de **LEITURA** na **view** `abt_empresa_total`.

Envia todas as queries em uma única string, separadas por `;`

In [62]:
sql_ex10 = """
CREATE USER 'caiomc_consult'@'%' IDENTIFIED BY '6b7997f42e0ebf3a51d2';
GRANT SELECT ON abt_empresa_total TO 'caiomc_consult'@'%';
"""

db(sql_ex10)

Executando query:


In [63]:
ia.sender(answer="sql_ex10", task="permissions", question="ex10", answer_type="pyvar")

interactive(children=(Button(description='Enviar ex10', style=ButtonStyle()), Output()), _dom_classes=('widget…

**Exercício 11**:

Foi criada uma aplicação que captura dados e faz a ingestão deles na base.

A política aceita pela empresa é de que a aplicação deve ter as permissões de:

- **INSERIR** e **LER** das tabelas da base `fiscamuni`:
    - empresa
    - fiscal
    - multa
    - propriedade
- **LER** das tabelas e views da base `fiscamuni`:
    - abt_empresa_total
    - motivo

Considere que a aplicação utiliza um usuário `u_ingest_multa` já existente com permissão de login de **qualquer host** e senha `e7854285319f1c83fcd1`.

Envia todas as queries em uma única string, separadas por `;`

In [68]:
sql_ex11 = """
GRANT SELECT, INSERT ON fiscamuni.empresa TO 'u_ingest_multa'@'%';
GRANT SELECT, INSERT ON fiscamuni.fiscal TO 'u_ingest_multa'@'%';
GRANT SELECT, INSERT ON fiscamuni.multa TO 'u_ingest_multa'@'%';
GRANT SELECT, INSERT ON fiscamuni.propriedade TO 'u_ingest_multa'@'%';
GRANT SELECT ON fiscamuni.abt_empresa_total TO 'u_ingest_multa'@'%';
GRANT SELECT ON fiscamuni.motivo TO 'u_ingest_multa'@'%';
FLUSH PRIVILEGES;

"""

db(sql_ex11)

Executando query:


ProgrammingError: 1410 (42000): You are not allowed to create a user with GRANT

In [69]:
ia.sender(answer="sql_ex11", task="permissions", question="ex11", answer_type="pyvar")

interactive(children=(Button(description='Enviar ex11', style=ButtonStyle()), Output()), _dom_classes=('widget…

**Exercício 12**:

Considere que o usuário `u_ingest_multa` tem todas as permissões relatadas no exercício anterior.

Ele não irá mais necessitar **INSERIR** na tabela `empresa`, nem **LER**/**INSERIR** na tabela `fiscal`.

Faça as atualizações, revogando as permissões não mais necessárias.

In [70]:
sql_ex12 = """
REVOKE INSERT ON fiscamuni.empresa FROM 'u_ingest_multa'@'%';
REVOKE SELECT, INSERT ON fiscamuni.fiscal FROM 'u_ingest_multa'@'%';
FLUSH PRIVILEGES;
"""

db(sql_ex12)

Executando query:


ProgrammingError: 1147 (42000): There is no such grant defined for user 'u_ingest_multa' on host '%' on table 'empresa'

In [71]:
ia.sender(answer="sql_ex12", task="permissions", question="ex12", answer_type="pyvar")

interactive(children=(Button(description='Enviar ex12', style=ButtonStyle()), Output()), _dom_classes=('widget…

**Exercício 13**:

O consultor (o mesmo dos exercícios anteriores) necessita de acesso a mais dados.

Porém, a empresa não quer liberar o acesso completo à base (contento o nome dos fiscais, nome das propriedades multadas e demais informações sensíveis).

**a)** Que solução você utilizaria neste caso?

<div class="alert alert-success">

Sua resposta AQUI!

</div>

**b)** Vamos supor que retornar os IDs não seja suficiente. Isto ocorre, por exemplo, quando você quer retornar o endereço de alguém, permitindo que o analista identifique que são endereços diferentes, mas sem saber exatamente qual rua.

Note que podemos ter muitas pessoas com o mesmo endereço(ex: mesma rua), e todas devem estar com o mesmo valor no campo.

Para este caso, uma sugestão é aplicar uma função de HASH, como SHA256.

Crie uma **view** `propriedade_consult` que contenha o `id` e o SHA256 da `descricao`, `cidade` e `endereco` da tabela `propriedade`. Mantenha o nome original das colunas.

Assim, o usuário utilizado pela consultoria poderia ter permissão de leitura apenas na **view** `propriedade_consult` e não na tabela original. Esta parte não precisa fazer, se quiser, teste localmente! 

In [75]:
sql_ex13b = """
CREATE VIEW propriedade_consult AS
SELECT
    id_propriedade,
    SHA2(descricao, 256) AS descricao,
    SHA2(cidade, 256) AS cidade,
    SHA2(endereco, 256) AS endereco
FROM fiscamuni.propriedade;
"""

db(sql_ex13b)

Executando query:


In [76]:
ia.sender(answer="sql_ex13b", task="permissions", question="ex13b", answer_type="pyvar")

interactive(children=(Button(description='Enviar ex13b', style=ButtonStyle()), Output()), _dom_classes=('widge…

**Exercício 14**:

Considere o retorno de uma coluna de **CPF** uma tabela utilizando SHA256.

Por exemplo:
```mysql
SELECT SHA2('377.662.560-02', 256)
UNION
SELECT SHA2('404.483.920-46', 256)
UNION
SELECT SHA2('196.499.400-49', 256)
UNION
SELECT SHA2('895.322.380-69', 256);
```

Analise a seguinte afirmação: Tendo os hashs e sabendo que é um campo de CPF, é impossível descobrir os CPFs originais.

<div class="alert alert-success">

Sua resposta AQUI!

</div>

**Exercício 15**:

Pesquise sobre anonimização de dados e mascaramento de dados. Explique a importância e como funciona.

<div class="alert alert-success">

Sua resposta AQUI!

</div>

**Exercício 16**:

Esqueça o usuário do banco de dados nesta questão, uma vez que ele é o usuário utilizado pelas aplicações em deploy e engenheiros da empresa.

Suponha que você foi contratado para criar uma aplicação que necessita de **login**. Os usuário devem possuir, pelo menos os campos de `id`, `nickname` e `senha`.

**a)** Construa a DDL de criação da tabela de `usuario`.

<div class="alert alert-success">

```mysql
-- SUA DDL aqui!
```
</div>

**b)** Qual seria a query para realizar uma inserção de um usuário nesta tabela? Utilize dados fictícios!

<div class="alert alert-success">

```mysql
-- SUA DML aqui!
```
</div>

**c)** Como você armazenou a senha no exercício "b)"? Você deixou como *plain-text*? Se sim, explique se foi uma boa escolha e quais as consequências!

<div class="alert alert-success">

Sua resposta AQUI!

</div>

**d)** Quais seriam alternativas melhores para armanenar dados sensíveis (como senhas) em banco de dados?

<div class="alert alert-success">

Sua resposta AQUI!

</div>

**e)** Pesquise sobre *Salting & peppering passwords*. Explique como funciona!

<div class="alert alert-success">

Sua resposta AQUI!

</div>

**f)** Pesquise sobre *senhas e entropia*. Anote abaixo os principais aprendizados!

<div class="alert alert-success">

Sua resposta AQUI!

</div>

### Fechando a conexão

In [ ]:
root_connection.close()

## Conferir Notas

Confira se as notas na atividade são as esperadas!

In [77]:
ia.grades(by="task", task="permissions")

|    | Tarefa      |   Nota |
|---:|:------------|-------:|
|  0 | permissions |     10 |

In [78]:
ia.grades(task="permissions")

|    | Atividade   | Exercício   |   Peso |   Nota |
|---:|:------------|:------------|-------:|-------:|
|  0 | permissions | ex01        |      1 |     10 |
|  1 | permissions | ex02        |      1 |     10 |
|  2 | permissions | ex03        |      1 |     10 |
|  3 | permissions | ex04        |      1 |     10 |
|  4 | permissions | ex05        |      1 |     10 |
|  5 | permissions | ex06        |      1 |     10 |
|  6 | permissions | ex07        |      1 |     10 |
|  7 | permissions | ex08        |      1 |     10 |
|  8 | permissions | ex09        |      1 |     10 |
|  9 | permissions | ex10        |      1 |     10 |
| 10 | permissions | ex11        |      1 |     10 |
| 11 | permissions | ex12        |      1 |     10 |
| 12 | permissions | ex13b       |      1 |     10 |

In [79]:
ia.grades(by="task")

|    | Tarefa               |   Nota |
|---:|:---------------------|-------:|
|  0 | agg_join             |   10   |
|  1 | ai_md_23_1           |   10   |
|  2 | ai_md_23_2           |   10   |
|  3 | ai_md_25_1           |    7.6 |
|  4 | ddl                  |   10   |
|  5 | desafio_normalizacao |    6   |
|  6 | dml                  |   10   |
|  7 | group_having         |   10   |
|  8 | newborn              |   10   |
|  9 | permissions          |   10   |
| 10 | select01             |   10   |
| 11 | sql_review1          |   10   |
| 12 | views                |   10   |

Até a próxima aula!